In [1]:
import os
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import numpy as np
from PIL import Image
import glob

from data_loader import RescaleT
from data_loader import CenterCrop
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import BASNet

def normPRED(d):
    ma = torch.max(d)
    mi = torch.min(d)

    dn = (d-mi)/(ma-mi)

    return dn

def save_output(image_name,pred,d_dir):

    predict = pred
    predict = predict.squeeze()
    predict_np = predict.cpu().data.numpy()

    im = Image.fromarray(predict_np*255).convert('RGB')
    img_name = image_name.split("\\")[-1]
    image = io.imread(image_name)
    imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

    pb_np = np.array(imo)

    aaa = img_name.split(".")
    bbb = aaa[0:-1]
    imidx = bbb[0]
    for i in range(1,len(bbb)):
        imidx = imidx + "." + bbb[i]

    imo.save(d_dir+imidx+'.png')


In C:\Users\yangz\miniconda3-2\envs\p36_2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\yangz\miniconda3-2\envs\p36_2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\yangz\miniconda3-2\envs\p36_2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\yangz\miniconda3-2\envs\p36_2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\yangz\miniconda3-2\envs\p36_2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed 

In [2]:
imgDir = 'C:/Users/yangz/Desktop/FireProjectBatchDownloadNew/PerimeterPreProcessedRescaled/PerimeterProcessedR'
predDir = 'C:/Users/yangz/Desktop/FireProjectBatchDownloadNew/PerimeterPostProcessed/I-70K/'
modelDir = 'C:/Users/yangz/Desktop/FireProject/BASNet/saved_models/basnet_bsi-Copy1/basnet_bsi_itr_70000_train_4.240915_tar_0.429964.pth'

imgNameList = glob.glob(imgDir + '*.jpg')

salObjDataset = SalObjDataset(img_name_list = imgNameList, lbl_name_list = [],transform=transforms.Compose([RescaleT(256),ToTensorLab(flag=0)]))
salObjLoader = DataLoader(salObjDataset, batch_size=1,shuffle=False,num_workers=1)

print("...load BASNet...")
net = BASNet(3,1)
net.load_state_dict(torch.load(modelDir))
if torch.cuda.is_available():
    net.cuda()
net.eval()

for i, data in enumerate(salObjLoader):

    print("inferencing:",imgNameList[i].split("/")[-1])

    inputs = data['image']
    inputs = inputs.type(torch.FloatTensor)

    if torch.cuda.is_available():
        inputs = Variable(inputs.cuda())
    else:
        inputs = Variable(inputs)

    d1,d2,d3,d4,d5,d6,d7,d8 = net(inputs)

    pred = d1[:,0,:,:]
    pred = normPRED(pred)

    save_output(imgNameList[i],pred,predDir)

    del d1,d2,d3,d4,d5,d6,d7,d8


...load BASNet...
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR0.jpg


C:\Users\yangz\miniconda3-2\envs\p36_2\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
C:\Users\yangz\miniconda3-2\envs\p36_2\lib\site-packages\torch\nn\functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
C:\Users\yangz\miniconda3-2\envs\p36_2\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR10.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR100.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1000.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1001.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1002.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1003.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1004.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1005.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1006.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1007.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1008.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR1009.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR101.jpg
inferencing: 

inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR151.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR152.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR153.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR154.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR155.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR156.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR157.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR158.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR159.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR16.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR160.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR161.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR162.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR163.jpg
inferencing: Perimete

inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR258.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR259.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR26.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR260.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR261.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR262.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR263.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR264.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR265.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR266.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR267.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR268.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR269.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR27.jpg
inferencing: Perimeter

...

inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR471.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR472.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR473.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR474.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR475.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR476.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR477.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR478.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR479.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR48.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR480.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR481.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR482.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR483.jpg
inferencing: Perimete

inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR579.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR58.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR580.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR581.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR582.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR583.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR584.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR585.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR586.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR587.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR588.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR589.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR59.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR590.jpg
inferencing: Perimeter

inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR685.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR686.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR687.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR688.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR689.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR69.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR690.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR691.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR692.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR693.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR694.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR695.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR696.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR697.jpg
inferencing: Perimete

inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR791.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR792.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR793.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR794.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR795.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR796.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR797.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR798.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR799.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR8.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR80.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR800.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR801.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR802.jpg
inferencing: PerimeterP

inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR898.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR899.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR9.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR90.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR900.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR901.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR902.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR903.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR904.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR905.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR906.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR907.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR908.jpg
inferencing: PerimeterPreProcessedRescaled\PerimeterProcessedR909.jpg
inferencing: PerimeterP